## Example for environment

In [1]:
from python_reinforcement_learning.gym_environment.project_hunter_environment import HunterEnvironment

env = HunterEnvironment()
env.reset()

observation, reward, terminated, truncated, info = env.step(0)
observation, reward, terminated, truncated, info = env.step(0)
observation, reward, terminated, truncated, info = env.step(0)

env.close()

print(observation)

HunterEnvironment initialized
{'player': array([0, 0]), 'enemies': array([0, 0])}


In [ ]:
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from python_reinforcement_learning.gym_environment.project_hunter_environment import HunterEnvironment
from ray.tune.logger import pretty_print

ray.init() # For easier debugging local_mode=True

config = (
        PPOConfig()
        .environment(HunterEnvironment, env_config={"size": 100, "max_timestep": 1000, "udp_address": "localhost:1337"})
        .framework("torch")
        .rollouts(num_rollout_workers=1, num_envs_per_worker=1) # num_envs_per_worker=2 for vectorization
        .resources(num_gpus=1)
    )

stop = {
    "training_iteration": 1,
    "timesteps_total": 10, # TODO
    "episode_reward_mean": 10, # TODO
}

algo = config.build()

for _ in range(stop["training_iteration"]):
    result = algo.train()
    print(pretty_print(result))
    # stop training of the target train steps or reward are reached
    if (
        result["timesteps_total"] >= stop["timesteps_total"]
        or result["episode_reward_mean"] >= stop["episode_reward_mean"]
    ):
        break
    
algo.stop()
        
ray.shutdown()

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig

config = (  # 1. Configure the algorithm,
    PPOConfig()
    .environment("Taxi-v3")
    .rollouts(num_rollout_workers=1)
    .framework("torch")
    .training(model={"fcnet_hiddens": [64, 64]})
    .evaluation(evaluation_num_workers=1)
    .resources(num_gpus=1)
)

algo = config.build()  # 2. build the algorithm,

for _ in range(1):
    print(algo.train())  # 3. train it,

algo.evaluate()  # 4. and evaluate it.